In [95]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [96]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [97]:
## Preprocess the data 
## drop irrelevant columns 
data = data.drop(['RowNumber', 'CustomerId', 'Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [98]:
## Encode categorical variable 
label_encoder_gender =LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [99]:
## One hot encode 'Geograph 
from sklearn.preprocessing import OneHotEncoder

onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [100]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [101]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [102]:
### combine one hot encoder columns with the original data 
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [103]:
#### save the encoders and scaler 
with open ('label_encoder_gender_regressor.pkl','wb') as file :
    pickle.dump(label_encoder_gender,file)
    
with open('onehot_encoder_geo_regressor.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [104]:
## divide the dataset into indepent and dependent features 
X = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

## split the data in tranning and testing set 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
## scale the features
scaler = StandardScaler()
X_train =scaler.fit_transform(X_train)
X_test =scaler.transform(X_test)
with open('scaler_regressor.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Regression problem statement 


In [105]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [106]:
##Build the model 
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])

In [107]:
## compile the model 
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                832       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [108]:
## setup the tensorboard 
log_dir = "reglogs/fit" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)


In [109]:
## setup early stopping 
early_stopping_callbacks = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [110]:
## Tranning the model 
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callbacks]
)

Epoch 1/100
250/250 [==============================] - 0s 896us/step - loss: 100125.2109 - mae: 100125.2109 - val_loss: 99578.1328 - val_mae: 99578.1328
Epoch 2/100
250/250 [==============================] - 0s 662us/step - loss: 99450.8125 - mae: 99450.8125 - val_loss: 98202.1719 - val_mae: 98202.1719
Epoch 3/100
250/250 [==============================] - 0s 645us/step - loss: 97046.7344 - mae: 97046.7344 - val_loss: 94656.8203 - val_mae: 94656.8203
Epoch 4/100
250/250 [==============================] - 0s 634us/step - loss: 92262.4766 - mae: 92262.4766 - val_loss: 88669.3438 - val_mae: 88669.3438
Epoch 5/100
250/250 [==============================] - 0s 626us/step - loss: 85176.2500 - mae: 85176.2500 - val_loss: 80670.6094 - val_mae: 80670.6094
Epoch 6/100
250/250 [==============================] - 0s 625us/step - loss: 76671.7266 - mae: 76671.7266 - val_loss: 72033.0703 - val_mae: 72033.0703
Epoch 7/100
250/250 [==============================] - 0s 626us/step - loss: 68127.2969 - ma

In [111]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [113]:
%tensorboard --logdir reglogs/fit


In [117]:
## Evaluate model on the test data 
test_loss,test_mae = model.evaluate(X_test,y_test)
print(f"test mae :{test_mae}")

63/63 [==============================] - 0s 373us/step - loss: 49919.8867 - mae: 49919.8867
test mae :49919.88671875


In [118]:
model.save('regression.h5')

/Users/sandipmahata/Documents/my self/krish naik  course /End ot End deeplearning project using ANN/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
